In [1]:
import pandas as pd

from src import utils
from src import pre_processing

In [2]:
# lendo e pré-processando os dados (json to dataframe)
data = pre_processing.pre_processing_tv_dataset('../datasets/tv/data.json')
data.head(1)

,tokens,aspect_tags
0,"[Excelente, smart, tv, ., E, foi, entregue, ve...","[0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"


In [3]:
# contabilizando o número de aspectos por registro
data = utils.aspect_counter(data)
data.head(1)

,tokens,aspect_tags,total_aspects
0,"[Excelente, smart, tv, ., E, foi, entregue, ve...","[0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",1


In [4]:
# definindo se um registro possui aspecto ou não
data = utils.has_aspect(data)
data.head(1)

,tokens,aspect_tags,total_aspects,has_aspect
0,"[Excelente, smart, tv, ., E, foi, entregue, ve...","[0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",1,1


In [5]:
# review
data['review'] = data.index + 1

In [6]:
# contabilizando reviews com aspects
has_aspect = data.has_aspect.value_counts(normalize=True).reset_index().rename(
    columns={'has_aspect': 'original_dataset', 'index': 'has_aspect'})
has_aspect

,has_aspect,original_dataset
0,1,0.906508
1,0,0.093492


In [7]:
# nome das colunas para estraficacão
y_col = 'has_aspect'
X_cols = ['tokens', 'aspect_tags', 'total_aspects']
groups = 'review'

In [8]:
# gerando os folds
data = utils.stratified_k_fold(data, X_cols, y_col, groups, k=10)

In [9]:
# número de registros por fold
data.fold.value_counts().reset_index()

,index,fold
0,7,110
1,1,109
2,2,109
3,3,109
4,4,109
5,5,109
6,6,109
7,8,109
8,9,109
9,10,109


In [10]:
# salvando
# data.to_csv('../datasets/processed/tv_stratified.csv', index=False)

In [11]:
# lendo a base salva
data = pd.read_csv('../datasets/processed/tv_stratified.csv')

In [12]:
# gerando estatísticas dos folds
for fold in data.fold.unique():

    # filtrando
    curr = data[data.fold == fold]

    # contabilizando a quantidade de registros por número de aspectos na frase
    curr_has_aspect = curr.has_aspect.value_counts(normalize=True).reset_index().rename(
        columns={'has_aspect': f'fold_{fold}', 'index': 'has_aspect'})
    has_aspect = has_aspect.merge(curr_has_aspect, how='outer')

In [13]:
# quantidade de registros por número de aspectos na frase
has_aspect = utils.fold_summary(has_aspect)
has_aspect

,has_aspect,original_dataset,fold_1,fold_2,fold_3,fold_4,fold_5,fold_6,fold_7,fold_8,fold_9,fold_10,fold_avg,fold_std
0,1,90.65,90.0,90.83,90.83,90.83,90.83,90.83,90.83,90.83,90.83,89.91,90.65,0.35
1,0,9.35,10.0,9.17,9.17,9.17,9.17,9.17,9.17,9.17,9.17,10.09,9.35,0.35
